In [50]:
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np

import math
import os

import tensorflow as tf
print(tf.__version__)

2.12.0


In [55]:
model_gs_path="gs://demo-tuned-tf-incident-pongthorn/model_df_tf"
map_severity_to_class={0:4,1: 3, 2: 2, 3: 1}
option=2
if option==1:
    unusedCols_unseen=['id','severity_name','open_to_close_hour','response_to_resolved_hour']
else:
    unusedCols_unseen=['id','severity_name','range_open_to_close_hour','range_response_to_resolved_hour']

# Load Model

In [56]:
abc_model = tf.keras.models.load_model(model_gs_path)  

[INFO 23-07-01 18:09:04.6912 UTC kernel.cc:1242] Loading model from path gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets/ with prefix b7df4480ba60471f
[INFO 23-07-01 18:09:05.3550 UTC decision_forest.cc:660] Model loaded with 76 root(s), 3878 node(s), and 7 input feature(s).
[INFO 23-07-01 18:09:05.3559 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-07-01 18:09:05.3565 UTC kernel.cc:1074] Use fast generic engine


# Make Prediction

In [60]:
new_data_table_id=f"{projectId}.{dataset_id}.new2_incident"

query_result=client.query(f"SELECT * FROM {new_data_table_id}")
unseen=query_result.to_dataframe()

unseen =unseen.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   severity_id                25 non-null     Int64  
 1   sla                        25 non-null     object 
 2   product_type               25 non-null     object 
 3   brand                      25 non-null     object 
 4   service_type               25 non-null     object 
 5   incident_type              25 non-null     object 
 6   open_to_close_hour         25 non-null     float64
 7   response_to_resolved_hour  25 non-null     float64
dtypes: Int64(1), float64(2), object(5)
memory usage: 1.7+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour
15,2,24x7 4Hrs Resolution Time,Server,HPE,Incident,Memory Failure,57.316667,0.000000
16,2,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,329.316667,329.300000
17,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,0.600000,0.583333
18,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,12.583333,12.583333
19,2,24x7 4Hrs Resolution Time,Software,Microsoft,Incident,Software,17.266667,15.166667
20,3,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,9.750000,1.233333
21,3,24x7 6Hrs Resolution Time,Software,SAPB1,Request,Software,150.000000,76.366667
22,2,9x5 NBD 4Hrs Response Time,Server,HPE,Incident,Memory Failure,2878.000000,0.500000
23,2,9x5 NBD 4Hrs Response Time,Server,HPE,Incident,Memory Failure,2878.000000,0.500000
24,2,9x5 NBD 4Hrs Response Time,Server,HPE,Incident,General Incident,4894.666667,0.000000


In [61]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
unseen_ds

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'response_to_resolved_hour': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}>

In [62]:
predResultList=abc_model.predict(unseen_ds)

for predResult in predResultList:
    _class=tf.argmax(predResult,-1).numpy()
    print(f"{predResult} : {_class} as severity#{map_severity_to_class[_class]}")   

1/1 [==============================] - 0s 23ms/step
[0.12458684 0.27730078 0.5568445  0.04126785] : 2 as severity#2
[0.0561542  0.28682926 0.39496237 0.26205423] : 2 as severity#2
[0.05830425 0.259523   0.41008484 0.27208787] : 2 as severity#2
[0.05163502 0.25586787 0.58655477 0.10594236] : 2 as severity#2
[0.07665343 0.09465248 0.4437611  0.3849329 ] : 2 as severity#2
[0.04009691 0.5711386  0.34551597 0.04324858] : 1 as severity#3
[0.03908427 0.65540355 0.26227185 0.04324043] : 1 as severity#3
[0.03911032 0.6662627  0.25307414 0.04155282] : 1 as severity#3
[0.04372857 0.6302673  0.28063557 0.04536868] : 1 as severity#3
[0.83008295 0.13320114 0.02023761 0.01647828] : 0 as severity#4
[0.8405155  0.12625903 0.01857574 0.01464964] : 0 as severity#4
[0.03562729 0.2171825  0.7095243  0.03766594] : 2 as severity#2
[0.04688214 0.2732623  0.60661834 0.07323733] : 2 as severity#2
[0.05026319 0.26938713 0.5980158  0.08233388] : 2 as severity#2
[0.04688214 0.2732623  0.60661834 0.07323733] : 2 as

2023-07-01 18:10:13.231427: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [25]
	 [[{{node Placeholder/_1}}]]
